<!DOCTYPE html>
<html lang="fa", dir="rtl">
<body >
    <div style="direction: rtl; unicode-bidi: embed;">
        <h1>محاسبات علمی با SciPy</h1>
        <p>کتابخانه <b>SciPy</b> (سای‌پای) قدرتمندترین ابزار پایتون برای محاسبات علمی و مهندسی است. این کتابخانه بر پایه NumPy ساخته شده و ابزارهایی برای حل مسائل پیچیده مانند انتگرال‌گیری، حل معادلات دیفرانسیل، پردازش سیگنال و آمار فراهم می‌کند.</p>
        <br>
        <b>نصب:</b> (معمولاً همراه با Anaconda نصب است)
        <pre>pip install scipy</pre>
    </div>
</body>
</html>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize, integrate, stats, signal

<!DOCTYPE html>
<html lang="fa", dir="rtl">
<body >
    <div style="direction: rtl; unicode-bidi: embed;">
        <h2>۱. برازش منحنی (Curve Fitting)</h2>
        <p>در آزمایشگاه، شما معمولاً نقاطی از داده دارید (مثلاً ولتاژ در برابر جریان) که دارای نویز هستند. هدف این است که بهترین خط یا منحنی ریاضی را پیدا کنید که از میان این نقاط می‌گذرد.</p>
        <p>ما از <code>scipy.optimize.curve_fit</code> استفاده می‌کنیم.</p>
    </div>
</body>
</html>

In [ ]:
# 1. تعریف تابعی که انتظار داریم داده‌ها از آن پیروی کنند (مثلاً خطی: y = ax + b)
def test_func(x, a, b):
    return a * x + b

# 2. تولید داده‌های آزمایشگاهی فرضی (با نویز تصادفی)
x_data = np.linspace(0, 10, 20)
y_exact = 2.5 * x_data + 1.0  # فرمول واقعی: a=2.5, b=1.0
noise = np.random.normal(0, 1.5, size=len(x_data))  # نویز
y_noisy = y_exact + noise

# 3. استفاده از SciPy برای پیدا کردن a و b
params, covariance = optimize.curve_fit(test_func, x_data, y_noisy)
a_fit, b_fit = params

print(f"پارامترهای واقعی: a=2.5, b=1.0")
print(f"پارامترهای تخمین زده شده: a={a_fit:.2f}, b={b_fit:.2f}")

# 4. رسم نمودار
plt.scatter(x_data, y_noisy, label='داده آزمایشگاه (با نویز)', color='red')
plt.plot(x_data, test_func(x_data, a_fit, b_fit), label='برازش (Fit)', color='blue', linewidth=2)
plt.legend()
plt.grid(True)
plt.show()

<!DOCTYPE html>
<html lang="fa", dir="rtl">
<body >
    <div style="direction: rtl; unicode-bidi: embed;">
        <h2>۲. حل معادلات دیفرانسیل (ODEs)</h2>
        <p>بسیاری از سیستم‌های فیزیکی (مثل افتادن جسم، مدار RC، سیستم فنر-جرم) با معادلات دیفرانسیل توصیف می‌شوند.
        تابع <code>solve_ivp</code> (حل مسئله مقدار اولیه) ابزار اصلی برای این کار است.</p>
        <p><b>مثال:</b> قانون سرمایش نیوتن. تغییر دما متناسب است با اختلاف دمای جسم و محیط:<br>
        dT/dt = -k * (T - T_env)</p>
    </div>
</body>
</html>

In [ ]:
# پارامترها
k = 0.3      # ضریب انتقال حرارت
T_env = 25   # دمای محیط (درجه سانتی‌گراد)
T0 = 100     # دمای اولیه جسم (آب جوش)

# تعریف تابع مشتق (dT/dt)
def cooling_law(t, T):
    return -k * (T - T_env)

# حل معادله
t_span = (0, 20)  # از ثانیه 0 تا 20
sol = integrate.solve_ivp(cooling_law, t_span, [T0], t_eval=np.linspace(0, 20, 100))

# رسم نتیجه
plt.plot(sol.t, sol.y[0])
plt.xlabel('Time (s)')
plt.ylabel('Temperature (°C)')
plt.title('Cooling of a Hot Object')
plt.grid()
plt.show()

<!DOCTYPE html>
<html lang="fa", dir="rtl">
<body >
    <div style="direction: rtl; unicode-bidi: embed;">
        <h2>۳. انتگرال‌گیری عددی</h2>
        <p>اگر تابعی دارید و می‌خواهید مساحت زیر نمودار آن را حساب کنید (انتگرال معین)، از <code>scipy.integrate.quad</code> استفاده می‌کنیم.</p>
    </div>
</body>
</html>

In [ ]:
# تابعی که می‌خواهیم انتگرال بگیریم: f(x) = x^2
def f(x):
    return x**2

# محاسبه انتگرال از 0 تا 2
result, error = integrate.quad(f, 0, 2)

print(f"انتگرال x^2 از 0 تا 2 برابر است با: {result:.4f}")
print(f"(مقدار واقعی: 2^3 / 3 = 2.6667)")

<!DOCTYPE html>
<html lang="fa", dir="rtl">
<body >
    <div style="direction: rtl; unicode-bidi: embed;">
        <h2>۴. پردازش سیگنال (Basic Signal Processing)</h2>
        <p>مهندسین برق دائماً با سیگنال‌های نویزی سر و کار دارند. بیایید یک سیگنال نویزی بسازیم و آن را صاف (Smooth) کنیم.</p>
    </div>
</body>
</html>

In [ ]:
# ساخت سیگنال
t = np.linspace(0, 10, 500)
signal_pure = np.sin(t)                   # سیگنال اصلی مثلی
noise = np.random.normal(0, 0.2, len(t))  # نویز سفید
signal_noisy = signal_pure + noise        # سیگنال آلوده به نویز

# استفاده از فیلتر Low-pass ساده (Savgol Filter)
signal_smooth = signal.savgol_filter(signal_noisy, window_length=51, polyorder=3)

# رسم
plt.figure(figsize=(10, 4))
plt.plot(t, signal_noisy, label='Noisy Signal', alpha=0.5, color='orange')
plt.plot(t, signal_smooth, label='Filtered Signal', color='black', linewidth=2)
plt.legend()
plt.title("Noise Removal")
plt.show()